<a href="https://colab.research.google.com/github/teixeirabrenno/agente-IA/blob/main/agenteIA-fornecedores.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [99]:
import os
from google.colab import userdata
from IPython.display import HTML, Markdown
from google import genai
from google.adk.agents import Agent
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.adk.tools import google_search
from google.genai import types  # Para criar conteúdos (Content e Part)
from datetime import date
import textwrap # Para formatar melhor a saída de texto
from IPython.display import display, Markdown # Para exibir texto formatado no Colab
import requests # Para fazer requisições HTTP
import warnings

warnings.filterwarnings("ignore")

os.environ["GOOGLE_API_KEY"] = userdata.get('GOOGLE_API_KEY')
client = genai.Client()
MODEL_ID = "gemini-2.0-flash"

In [100]:
# Função auxiliar que envia uma mensagem para um agente via Runner e retorna a resposta final
def call_agent(agent: Agent, message_text: str) -> str:
    # Cria um serviço de sessão em memória
    session_service = InMemorySessionService()
    # Cria uma nova sessão (você pode personalizar os IDs conforme necessário)
    session = session_service.create_session(app_name=agent.name, user_id="user1", session_id="session1")
    # Cria um Runner para o agente
    runner = Runner(agent=agent, app_name=agent.name, session_service=session_service)
    # Cria o conteúdo da mensagem de entrada
    content = types.Content(role="user", parts=[types.Part(text=message_text)])

    final_response = ""
    # Itera assincronamente pelos eventos retornados durante a execução do agente
    for event in runner.run(user_id="user1", session_id="session1", new_message=content):
        if event.is_final_response():
          for part in event.content.parts:
            if part.text is not None:
              final_response += part.text
              final_response += "\n"
    return final_response

In [101]:
# Função auxiliar para exibir texto formatado em Markdown no Colab
def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

In [102]:
##########################################
# --- Agente 1: Buscador de Fornecedores --- #
##########################################

def agente_buscador(topico, data_de_hoje, regiao):
  buscador = Agent(
      name='agente_buscador',
      model='gemini-2.0-flash',
      instruction = '''Você é um agente que busca informações de fornecedores de insumos alimentícios para uma Unidade de Alimentação e Nutrição (UAN).
      Seu dever é buscar os fornecedores, baseados na sua avaliação e custo-benefício. Dê prioridade ao custo-benefício, mas caso a avaliação da
      empresa fornecedora for abaixo de quatro estrelas, dê preferência a outra empresa, mesmo que o valor seja um pouco mais caro. Dê preferência
      a fornecedores próximos ao local da UAN.
      As informações e preços dos insumos devem estar atualizadas, com no máximo duas semanas antes da data de hoje.
      Mostre os 5 melhores fornecedores da região baseados nos critérios acima. Cite o endereço, telefone, WhatsApp e email de contato deles.''',
      description = 'Agente que busca informações sobre fornecedores de insumos alimentares no Google',
      tools = [google_search],
  )

  entrada_do_agente_buscador = f"Tópico: {topico}\nData de hoje: {data_de_hoje}\nLocalização: {local_de_busca}\nLocal da UAN: {local_UAN}"
  # Executa o agente
  informacoes_fornecedores = call_agent(buscador, entrada_do_agente_buscador)
  return informacoes_fornecedores


In [103]:
#Informações passadas ao agente

data_de_hoje = date.today().strftime("%d/%m/%Y")
local_de_busca = 'Rio de Janeiro - Rio de Janeiro'
local_UAN = 'Barra da Tijuca, Rio de Janeiro - RJ, 22640-100'
topico = input("Por favor, digite o insumo o qual você deseja pesquisar: ")


if not topico:
  print('Você não escolheu nenhum insumo ainda.')
else:
  print(f'Pesquisando sobre fornecedores de {topico.lower()}!')

  informacoes_fornecedores = agente_buscador(topico, data_de_hoje, local_de_busca)


  print('------- Resultado do Agente Bsucador --------')
  display(to_markdown(informacoes_fornecedores))
  print('--------------------------------------------------')





Por favor, digite o insumo o qual você deseja pesquisar: batata
Pesquisando sobre fornecedores de batata!
------- Resultado do Agente Bsucador --------


> Para encontrar os melhores fornecedores de batata para sua UAN na Barra da Tijuca, Rio de Janeiro, vou realizar uma pesquisa com base na proximidade, custo-benefício e avaliações dos fornecedores. Priorizarei fornecedores com boa avaliação (acima de 4 estrelas) e informações de contato atualizadas.
> 
> 
> Com base nas informações encontradas, aqui estão 5 fornecedores que podem atender às suas necessidades de batata na Barra da Tijuca, Rio de Janeiro:
> 
> 1.  **Líder Rio Distribuidora:**
>     *   **O que oferece:** Grande variedade de batatas congeladas (Bem Brasil, McCain, Quality).
>     *   **Endereço:** Rua Oliveiros Rodrigues Alves, 2063, Cerâmica - Nova Iguaçu - RJ.
>     *   **Telefone:** (21) 3262-1162
>     *   **Email:** [lider@liderrioalimentos.com.br](mailto:lider@liderrioalimentos.com.br)
>     *   **Observações:** Possuem uma vasta gama de produtos alimentícios e experiência no mercado.
> 
> 2.  **Formoso Carnes e Derivados Ltda:**
>     *   **O que oferece:** Batata congelada Mais Bem Brasil.
>     *   **Endereço:** Rua do Arroz, 90, Rio de Janeiro - RJ.
>     *   **WhatsApp:** (21) 96472-6143
>     *   **Email:** [adm@formosocarnes.com.br](mailto:adm@formosocarnes.com.br)
> 
> 3.  **Assaí Atacadista (Barra da Tijuca):**
>     *   **O que oferece:** Batata inglesa.
>     *   **Endereço:** Av. das Américas, 900 - Barra da Tijuca, Rio de Janeiro - RJ, 22793-081.
>     *   **Observações:** Possibilidade de entrega via Rappi.
> 
> 4.  **PR Comércio:**
>     *   **O que oferece:** Batata frita congelada.
>     *   **Endereço:** Av. Automóvel Clube, lote 01 - quadra 07 - Duque de Caxias - RJ.
>     *   **WhatsApp:** Fale por WhatsApp no site
>     *   **Observações:** Distribui para diversos segmentos como mercados e restaurantes.
> 
> 5.  **Batata Frita Popular:**
>     *   **O que oferece:** Batata frita.
>     *   **Endereço:** Rua do Catumbí, 88, Catumbí – Rio de Janeiro.
>     *   **Telefone:** (21) 2502-2152 / 2502-1614 / 2273-6612
>     *   **Email:** [popular@batatafritapopular.com.br](mailto:popular@batatafritapopular.com.br)
>     *   **Observações:** Fabricante com loja na fábrica.
> 
> **Outras opções:**
> 
> *   **Riobatata:** Comércio por grosso de batata e outros produtos (cebola, alho, azeitonas e gengibre). Possuem armazém em Freiria de Rio Maior e oferecem entregas. Contato: 966 136 750 / 966 011 392.
> *   **Armazém do Grão:** Possui uma unidade na Barra da Tijuca (Av. das Américas, 11.599). Telefone: (21) 3547-3046 / 3547-3047. WhatsApp: (24) 99934-2123.
> *   **Mercado Rappi - Barra da Tijuca 3:** Endereço: Av. das Américas, 7841 - Barra da Tijuca, Rio de Janeiro.
> 
> **Observações Adicionais:**
> 
> *   **HF Brasil:** Apresenta preços de atacado da batata no Rio de Janeiro, com dados atualizados até 16 de maio de 2025.
> 
> Recomendo verificar os preços e condições de cada fornecedor, bem como a qualidade dos produtos, antes de tomar uma decisão final.
> 


--------------------------------------------------
